In [ ]:
!git clone https://github.com/Parthiee/SemEval

Cloning into 'SemEval'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 112 (delta 49), reused 76 (delta 18), pack-reused 0
Receiving objects: 100% (112/112), 11.76 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
import pandas as pd

df = pd.read_csv("/content/SemEval/pd2.csv")




In [ ]:
emotion_mapping = {'disgust': 0, 'contempt': 1, 'anger': 2, 'neutral': 3, 'joy': 4, 'sadness': 5, 'fear': 6, 'surprise': 7}

# Map emotions to numerical labels in training data
df['target'] = df['emotion'].map(emotion_mapping)



In [ ]:
!pip install pyphonetics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 68.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pyphonetics import FuzzySoundex
from Levenshtein import distance
import re

# Function to create a dictionary with phonetic codes
def create_phonetic_dict(words):
    phonetic_dict = {}
    soundex = FuzzySoundex()

    for i in range(len(words)):
        # Clean the word by removing non-alphabetic characters
        cleaned_word = re.sub(r'[^a-zA-Z]', '', words[i])

        # Skip empty or very short cleaned words
        if not cleaned_word or len(cleaned_word) <= 1:
            continue

        try:
            phonetic_code = soundex.phonetics(cleaned_word)
            if not phonetic_code:
                continue

            last_letter = cleaned_word[-1]

            if phonetic_code not in phonetic_dict:
                phonetic_dict[phonetic_code] = {last_letter: {cleaned_word}}
            else:
                if last_letter not in phonetic_dict[phonetic_code]:
                    phonetic_dict[phonetic_code][last_letter] = {cleaned_word}
                else:
                    phonetic_dict[phonetic_code][last_letter].add(cleaned_word)

        except Exception as e:
            print(f"Error processing word: {cleaned_word}, Original word: {words[i]}, Error: {e}")
            continue
    return phonetic_dict



# Function for spelling normalization
def normalize_spelling(input_word, phonetic_dict):
    soundex = FuzzySoundex()
    phonetic_code = soundex.phonetics(input_word)
    last_letter = input_word[-1]

    if phonetic_code in phonetic_dict and last_letter in phonetic_dict[phonetic_code]:
        potential_candidates = phonetic_dict[phonetic_code][last_letter]
        best_match = min(potential_candidates, key=lambda candidate: distance(input_word, candidate))
        return best_match

    return input_word


# Extract unique words from the 'utterance' column in your overall training data
unique_words = set(" ".join(df['utterance'].astype(str)).split())
unique_words=list(unique_words)
# Create a dictionary with phonetic codes
phonetic_dict = create_phonetic_dict(unique_words)


Error processing word: hhhhhh, Original word: hhhhhh, Error: string index out of range
Error processing word: ww, Original word: ww, Error: string index out of range
Error processing word: gh, Original word: gh, Error: string index out of range


In [ ]:
print(phonetic_dict)

{'N7457': {'a': {'nikalunga', 'nikaaluunga'}, 'i': {'nikalengi'}, 'e': {'nikalenge', 'nikaalenge'}}, 'S36': {'r': {'sudhar', 'swtor', 'starr', 'star', 'sidhar', 'sweater', 'sootr', 'sudhaar', 'sthir', 'stir', 'sahodar', 'sweethear', 'shooter'}, 'i': {'stri', 'suthri'}, 'a': {'satra', 'sootra', 'suthra', 'sutra', 'stariya', 'sitara'}, 'w': {'straw'}, 'e': {'store', 'sitaare', 'sidhaare', 'stree'}, 't': {'sweetheart', 'start'}, 'u': {'shatru'}, 'y': {'story'}}, 'G67': {'k': {'grahak', 'graahak'}, 'j': {'gairaj', 'garaj'}, 'e': {'girje'}, 'a': {'girega', 'georgia'}, 'i': {'giraogi', 'guruji'}}, 'C17': {'e': {'chipke', 'chupake', 'chupke'}, 'k': {'chhapaak', 'chipak', 'chabuk'}, 'u': {'chipku'}, 'i': {'chaubeji', 'chapagai'}, 'a': {'chipka'}}, 'K55': {'y': {'kaminey'}, 'n': {'kaamon', 'khaanon', 'kamaan', 'cannon', 'kaunaun', 'kanyaaen', 'kahaaniyan', 'konon', 'kaanon', 'comeon', 'kaanoon', 'keemun', 'kanon', 'kaamnaon', 'kahaaniyon', 'common'}, 'a': {'kameena', 'kamina', 'kaamana'}, 'e': 

In [ ]:
df.dropna()

,utterance,emotion,target
0,kya kya bhar ke rakha hai ne ghar mein please ...,disgust,0
1,ok chalo chalo bahar,contempt,1
2,momma hath chhodiye dad,anger,2
3,dekho ya na bol raha hai fir tum hi kahogi fal...,neutral,3
4,tum kitne salo se ye joke maar rahe ho koi hot...,neutral,3
...,...,...,...
25056,kaba taka aeka hii baat bologe,anger,2
25057,chup chaap merey paise lauta,anger,2
25058,tujhe aeka baar main samajh naheen aataa kyaa,anger,2
25059,bahut confucing interfaces haye tumhara,anger,2


In [ ]:
df.head()

,utterance,emotion,target
0,kya kya bhar ke rakha hai ne ghar mein please ...,disgust,0
1,ok chalo chalo bahar,contempt,1
2,momma hath chhodiye dad,anger,2
3,dekho ya na bol raha hai fir tum hi kahogi fal...,neutral,3
4,tum kitne salo se ye joke maar rahe ho koi hot...,neutral,3


In [ ]:
import pandas as pd
from pyphonetics import FuzzySoundex
from Levenshtein import distance
import re

# Function for spelling correction
def correct_spelling(input_word, phonetic_dict):
    soundex = FuzzySoundex()
    try:
        phonetic_code = soundex.phonetics(input_word)
        last_letter = input_word[-1]

        if phonetic_code in phonetic_dict and last_letter in phonetic_dict[phonetic_code]:
            potential_candidates = phonetic_dict[phonetic_code][last_letter]

            # Calculate Levenshtein distance for each potential candidate
            distances = {candidate: distance(input_word, candidate) for candidate in potential_candidates}

            # Choose the candidate with a Levenshtein distance below a certain threshold
            threshold = 3  # Adjust this threshold as needed
            best_match = min(distances, key=distances.get)

            # Return the best matching candidate
            return best_match
    except Exception as e:
        print(f"Error processing word: {input_word}, Original word: {input_word}, Error: {e}")

    return input_word

# Function to apply spelling correction to a text
def correct_text_spelling(text, phonetic_dict):
    words = re.findall(r'\b\w+\b', str(text))  # Extract words from the text
    corrected_words = [correct_spelling(word, phonetic_dict) for word in words]

    corrected_text = ' '.join(corrected_words)
    return corrected_text



# Apply spelling correction to all sentences in the 'utterance' column
df['corrected_utterance'] = df['utterance'].apply(lambda x: correct_text_spelling(x, phonetic_dict))

# Display the original and corrected utterances
print(df[['utterance', 'corrected_utterance']])


Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range
Error processing word: h, Original word: h, Error: string index out of range

In [ ]:
df.head(10)

,utterance,emotion,target,corrected_utterance
0,kya kya bhar ke rakha hai ne ghar mein please ...,disgust,0,kya kya bhar ke rakha hai ne ghar mein please ...
1,ok chalo chalo bahar,contempt,1,ok chalo chalo bahar
2,momma hath chhodiye dad,anger,2,momma hath chhodiye dad
3,dekho ya na bol raha hai fir tum hi kahogi fal...,neutral,3,dekho ya na bol raha hai fir tum hi kahogi fal...
4,tum kitne salo se ye joke maar rahe ho koi hot...,neutral,3,tum kitne salo se ye joke maar rahe ho koi hot...
5,kaikai pe joke maroge manthra thodi na hasegi,contempt,1,kaikai pe joke maroge manthra thodi na hasegi
6,funny,contempt,1,funny
7,aur aa rahe hain dekhna yahi jo un per istemal...,neutral,3,aur aa rahe hain dekhna yahi jo un per istemal...
8,hi guys,joy,4,hi guys
9,hi,joy,4,hi


In [ ]:
import pandas as pd
from pyphonetics import FuzzySoundex
from Levenshtein import distance
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Get the set of English stopwords
stopwords_set = set(stopwords.words('english'))
df['corrected_utterance'] = df['corrected_utterance'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords_set]))

# Display the original and corrected utterances
print(df[['utterance', 'corrected_utterance']])

                                               utterance  \
0      kya kya bhar ke rakha hai ne ghar mein please ...   
1                                   ok chalo chalo bahar   
2                                momma hath chhodiye dad   
3      dekho ya na bol raha hai fir tum hi kahogi fal...   
4      tum kitne salo se ye joke maar rahe ho koi hot...   
...                                                  ...   
25056                     kaba taka aeka hii baat bologe   
25057                       chup chaap merey paise lauta   
25058      tujhe aeka baar main samajh naheen aataa kyaa   
25059            bahut confucing interfaces haye tumhara   
25060                        mai tumse bahut krodhit hoo   

                                     corrected_utterance  
0      kya kya bhar ke rakha hai ne ghar mein please ...  
1                                   ok chalo chalo bahar  
2                                momma hath chhodiye dad  
3      dekho ya na bol raha hai fir tum hi 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head(20)

,utterance,emotion,target,corrected_utterance
0,kya kya bhar ke rakha hai ne ghar mein please ...,disgust,0,kya kya bhar ke rakha hai ne ghar mein please ...
1,ok chalo chalo bahar,contempt,1,ok chalo chalo bahar
2,momma hath chhodiye dad,anger,2,momma hath chhodiye dad
3,dekho ya na bol raha hai fir tum hi kahogi fal...,neutral,3,dekho ya na bol raha hai fir tum hi kahogi fal...
4,tum kitne salo se ye joke maar rahe ho koi hot...,neutral,3,tum kitne salo se ye joke maar rahe ho koi hot...
5,kaikai pe joke maroge manthra thodi na hasegi,contempt,1,kaikai pe joke maroge manthra thodi na hasegi
6,funny,contempt,1,funny
7,aur aa rahe hain dekhna yahi jo un per istemal...,neutral,3,aur aa rahe hain dekhna yahi jo un per istemal...
8,hi guys,joy,4,hi guys
9,hi,joy,4,hi


In [ ]:
import pandas as pd

# Your dictionary of names
names_to_remove = {'indravadan', 'amar twanshu', 'amar twanshu/behra', 'anirudh', 'baa', 'babu', 'bacche', 'bakasur', 'baldev', 'bhatia', 'chika', 'crowd', 'dharam paal singh', 'dinesh', 'doctors', 'don', 'dr. acharya', 'dushyant', 'dushyant  ', 'everyone', 'fufa', 'gurpreet', 'ila', 'ila ', 'ila ben', 'indravadan', 'indravadhan', 'indravandhan', 'indravardhan', 'indravardhan / maya', 'indu', 'indu ', 'indu & maya', 'indu and ila ', 'indu/gunga', 'inpector', 'inspector', 'jugalkishore', 'kiran', 'kiran ', 'kismi', 'krish', 'ma', 'madhu', 'madhu fufa', 'madhusudan', 'madhusudhan', 'mahesh', 'maid', 'maya', 'maya ', "maya's sister", 'maya/ sarupa', 'mom', 'monish', 'monisha', 'monisha ', 'monisha and indu', 'mr. sindolin', 'mrs. desai', 'musician', 'nagesh', 'naisha', 'neelima', 'pandit', 'patang/karan', 'person', 'person 1 ', 'person1', 'phir', 'pillu aunty', 'police', 'prabhavati', 'radha', 'radha bai', 'rajinder', 'ramba', 'ranjana', 'rita', 'rosesh', 'rosesh ', 'rosesh/andha', 'rosheh', 'roshesh', 'rupa mehta', 'sahi', 'sahil', 'sahil ', 'sahil & rosesh', 'sarita', 'sarupa', 'servant', 'sharman', 'simple', 'sita or geeta', 'sohail', 'soniya', 'sonya', 'stranger', 'sudhanshu', 'suketu', 'sunita', 'suri', 'tanvir', 'tanya', 'together', 'vetha', 'vidisha', 'vithal', 'vitthal', 'waiter', 'yamraj'}


# Remove names from the 'corrected_utterance' column
df['corrected_utterance'] = df['corrected_utterance'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in names_to_remove]))

# Display the original and corrected utterances
print(df[['utterance', 'corrected_utterance']])


                                               utterance  \
0      kya kya bhar ke rakha hai ne ghar mein please ...   
1                                   ok chalo chalo bahar   
2                                momma hath chhodiye dad   
3      dekho ya na bol raha hai fir tum hi kahogi fal...   
4      tum kitne salo se ye joke maar rahe ho koi hot...   
...                                                  ...   
25056                     kaba taka aeka hii baat bologe   
25057                       chup chaap merey paise lauta   
25058      tujhe aeka baar main samajh naheen aataa kyaa   
25059            bahut confucing interfaces haye tumhara   
25060                        mai tumse bahut krodhit hoo   

                                     corrected_utterance  
0      kya kya bhar ke rakha hai ne ghar mein please ...  
1                                   ok chalo chalo bahar  
2                                momma hath chhodiye dad  
3      dekho ya na bol raha hai fir tum hi 

In [ ]:
df.head(20)

,Unnamed: 0,utterance,emotion,target,corrected_utterance
0,0,kya kya bhar ke rakha hai indravadan ne is gha...,disgust,0,kya kya bhar ke rakha hai ne ghar mein please ...
1,1,"ok, chalo rosesh chalo bahar",contempt,1,ok chalo chalo bahar
2,2,momma! hath chhodiye dad!,anger,2,momma hath chhodiye dad
3,3,dekho maya ya na bol raha hai! fir tum hi kaho...,neutral,3,dekho ya na bol raha hai fir tum hi kahogi fal...
4,4,"indravadan, tum kitne salo se ye joke maar rah...",neutral,3,tum kitne salo se ye joke maar rahe ho koi hot...
5,5,kaikai pe joke maroge to manthra thodi na hasegi,contempt,1,kaikai pe joke maroge manthra thodi na hasegi
6,6,very funny,contempt,1,funny
7,7,"maya, monisha aur sahil aa rahe hain, dekhna y...",neutral,3,aur aa rahe hain dekhna yahi jo un per istemal...
8,8,hi guys,joy,4,hi guys
9,9,hi,joy,4,hi


In [ ]:
df['utterance']=df['corrected_utterance']

In [ ]:
df.head()

,Unnamed: 0,utterance,emotion,target,corrected_utterance
0,0,kya kya bhar ke rakha hai ne ghar mein please ...,disgust,0,kya kya bhar ke rakha hai ne ghar mein please ...
1,1,ok chalo chalo bahar,contempt,1,ok chalo chalo bahar
2,2,momma hath chhodiye dad,anger,2,momma hath chhodiye dad
3,3,dekho ya na bol raha hai fir tum hi kahogi fal...,neutral,3,dekho ya na bol raha hai fir tum hi kahogi fal...
4,4,tum kitne salo se ye joke maar rahe ho koi hot...,neutral,3,tum kitne salo se ye joke maar rahe ho koi hot...


In [ ]:
import pandas as pd

# Assuming df is your DataFrame with the 'utterance' column
df['utterance'] = df['utterance'].replace(to_replace=r'\d', value='', regex=True)

# Display the DataFrame after removing numbers
print(df)


                                               utterance   emotion  target  \
0      kya kya bhar ke rakha hai ne ghar mein please ...   disgust       0   
1                                   ok chalo chalo bahar  contempt       1   
2                                momma hath chhodiye dad     anger       2   
3      dekho ya na bol raha hai fir tum hi kahogi fal...   neutral       3   
4      tum kitne salo se ye joke maar rahe ho koi hot...   neutral       3   
...                                                  ...       ...     ...   
25056                     kaba taka aeka hii baat bologe     anger       2   
25057                       chup chaap merey paise lauta     anger       2   
25058      tujhe aeka baar main samajh naheen aataa kyaa     anger       2   
25059            bahut confucing interfaces haye tumhara     anger       2   
25060                        mai tumse bahut krodhit hoo     anger       2   

                                     corrected_utterance  
0   